# OpenSpiel + Gambit Workflow on one card poker

In this tutorial, we will:

1. Load examples of normal form and extensive form games in OpenSpiel and Gambit
2. Train agents in OpenSpiel to play games and create strategies
3. Compare results against equilibria computed with Gambit

This notebook demonstrates the workflow between OpenSpiel and Gambit for game-theoretic analysis:

- **OpenSpiel**: Provides iterative learning algorithms for strategy approximation
- **Gambit**: Provides exact equilibrium computation for theoretical comparison

In [37]:
import pygambit as gbt
import pyspiel
from open_spiel.python.egt.utils import game_payoffs_array
from open_spiel.python.egt import dynamics
import numpy as np

Load matrix rock-paper-scissors from OpenSpiel:

In [52]:
ops_matrix_rps_game = pyspiel.load_game("matrix_rps")

Get the payoffs as numpy arrays...

In [53]:
matrix_rps_payoffs = game_payoffs_array(ops_matrix_rps_game)
matrix_rps_payoffs

array([[[ 0., -1.,  1.],
        [ 1.,  0., -1.],
        [-1.,  1.,  0.]],

       [[ 0.,  1., -1.],
        [-1.,  0.,  1.],
        [ 1., -1.,  0.]]])

... which we can use to recreate the game in Gambit:

In [54]:
gbt_matrix_rps_game = gbt.Game.from_arrays(
    matrix_rps_payoffs[0],
    matrix_rps_payoffs[1],
    title="Matrix Rock-Paper-Scissors"
)
gbt_matrix_rps_game

,1,2,3
1,"0.0,0.0","-1.0,1.0","1.0,-1.0"
2,"1.0,-1.0","0.0,0.0","-1.0,1.0"
3,"-1.0,1.0","1.0,-1.0","0.0,0.0"


The equilibrium strategy for both players is to choose rock, paper, and scissors with equal probability:

In [57]:
gbt.nash.lcp_solve(gbt_matrix_rps_game).equilibria[0]

[[Rational(1, 3), Rational(1, 3), Rational(1, 3)], [Rational(1, 3), Rational(1, 3), Rational(1, 3)]]

In [40]:
dyn = dynamics.SinglePopulationDynamics(matrix_rps_payoffs, dynamics.replicator)
x = np.array([0.2, 0.2, 0.6]) # population heavily-weighted toward scissors
dyn(x)

array([ 0.08, -0.08,  0.  ])

## Step 2: Train Agents in OpenSpiel (CFR)

In [ ]:
# from open_spiel.python.algorithms import cfr

# cfr_solver = cfr.CFRSolver(game)

# for i in range(100):
#     cfr_solver.evaluate_and_update_policy()

# avg_policy = cfr_solver.average_policy()
# print("Sampled strategy:", avg_policy)

Sampled strategy: <open_spiel.python.policy.TabularPolicy object at 0x10a24ac50>


## Step 4: Load Game in Gambit

In [ ]:
# result = gbt.nash.lcp_solve(g)
# eqm = result.equilibria[0]
# eqm

[[[Rational(1, 1), Rational(0, 1)], [Rational(1, 3), Rational(2, 3)]], [[Rational(2, 3), Rational(1, 3)]]]

## Step 5: Compare Results